 # MTH8211 : Algèbre linéaire numérique appliquée
 ## Laboratoire 3 : Application de la SVD, discussions sur la performance, systèmes de point de selle
 
Geoffroy Leconte

Source images, signaux bruités: http://www.imm.dtu.dk/~pcha/HNO/ (Matrices, Spectra, and Filtering
Per Christian Hansen, James G. Nagy, and Dianne P. O'Leary)

## I) Application de la SVD: débruitage d'images

In [ ]:
# ]add Images, ToeplitzMatrices

In [ ]:
using Images, LinearAlgebra, ToeplitzMatrices

$X \in \mathbb{R}^{m \times n}$ représente l'image désirée.

$B \in \mathbb{R}^{m \times n}$ représente l'image bruitée.

Si le bruitage des lignes est indépendant de celui des colonnes, il existe $A_c \in \mathbb{R}^{m \times m}$ et $A_r \in \mathbb{R}^{n \times n}$.

$$A_c X A_r^T = B$$.

L'implémentation naive consiste à calculer:

$$X_{\textrm{naive}} = A_c^{-1} B A_r^{-T}$$

In [ ]:
function challenge1(m,n,noise)
    X = zeros(m,n)
    I = Int(round(m/5)):Int(round(3*m/5))
    J = Int(round(n/5)):Int(round(3*n/5))
    X[I,J] .= 0.5
    for i=1:m
     for j=1:n
       if (i - round(3 * m / 5))^2 + (j - round(5 * n / 8))^2 < round(max(m, n) / 5)^2
          X[i,j] = 1
       end
     end
    end
    c = zeros(m)
    c[1:5] .= 5:-1:1 ./ 15
    Ac = SymmetricToeplitz(c)
    c = zeros(n)
    c[1:5] .= 5:-1:1 ./ 15
    r = zeros(n)
    r[1:10] .= 5:-0.5:0.5 ./ 15
    Ar = Toeplitz(c, r)
    B = Ac * X * Ar' + noise * randn(m,n)
    return B, Ac, Ar, X
end
m = n = 256
noise = 0.01
B, Ac, Ar, X = challenge1(m, n, noise);

1) Calculer $X_{\textrm{naive}}$.

In [ ]:
Xnaive = (Ar \ (Ac \ B)')'
Gray.(Xnaive)
# Ac \ B / Ar'

2) Soit $E$ le bruit. $X_{\textrm{naive}} = X + A_c^{-1} E A_r^{-T}$. Calculez le bruit inversé $  A_c^{-1} E A_r^{-T}$ et comparez avec la valeur de $B$.

In [ ]:
E = B - X
NoiseInv = (Ar \ (Ac \ E)')'
norm(NoiseInv) / norm(B)

3) Effectuez une décomposition en valeurs singulières de $A_c$ et $A_r$. Utilisez la décomposition en valeur singulières pour réduire la contribution du bruit. Calculez l'image débruitée et comparez avez la solution exacte.

In [ ]:
Fc = svd(Matrix(Ac))
Fr = svd(Matrix(Ar))
n_svd = sum(abs.(Fc.S) .≥ 0.1)
X_debruit = Fc.V[1:end, 1:n_svd] * Diagonal(1 ./ Fc.S[1:n_svd]) * Fc.U[1:end, 1:n_svd]' * 
    B *
    Fr.U[1:end, 1:n_svd] * Diagonal(1 ./ Fr.S[1:n_svd]) * Fr.V[1:end, 1:n_svd]'
Gray.(X_debruit)

In [ ]:
Gray.(X)

## II) Retour plus en détail sur la performance et les fonctions de BLAS et LAPACK 

BLAS (Basic Linear Algebra Subprograms) est une suite de routines pour effectuer des opérations vectorielles/matricielles. Elles sont optimisées spécifiquement pour divers types de plateformes, ce qui les rend particulièrement performantes.

In [ ]:
# ]add BenchmarkTools

In [ ]:
using BenchmarkTools

In [ ]:
T = Float64
n = 10000
a = rand(T, n)
b = rand(T, n)
c = rand(T, n);

In [ ]:
@benchmark dot($a, $b) # interpoler a et b avec $ pour éviter le problème du benchmark avec des variables globales

In [ ]:
function my_custom_dot(a::Vector{T}, b::Vector{T}) where {T}
    res = zero(T)
    n = length(a)
    @assert n == length(b)
    for i=1:n
        res += a[i] * b[i]
    end
    return res 
end

In [ ]:
@benchmark my_custom_dot($a, $b)

In [ ]:
@code_lowered dot(a, b) # appel à BLAS.dot

Les fonctions de BLAS ne gèrent pas tous les types:

In [ ]:
# ]add DoubleFloats

In [ ]:
using DoubleFloats

In [ ]:
T = Double64 # plus précis que Float64
n = 10000
a = rand(T, n)
b = rand(T, n)
c = rand(T, n);

In [ ]:
@code_lowered dot(a, b) # retourne à l'implémentation en pur julia car non implémenté avec BLAS

In [ ]:
@benchmark BLAS.dot($a, $b)

In [ ]:
@benchmark dot($a, $b)

In [ ]:
@benchmark my_custom_dot($a, $b)

Julia utilise d'autres opérations de BLAS qui sont utilisées à travers d'autres fonctions courantes.

Les opérations d'algèbre linéaire dense sont effectuées avec la bibliothèque LAPACK, qui elle-même utilise BLAS.
Cette bibliothèque ne fonctionne qu'avec les types `Float32, Float64, ComplexF32, ComplexF64`.

Les fonction de LAPACK disponibles dans la [documentation de Julia](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LAPACK-functions) ne sont pas vouées à être utilisées directement. Elle seront appelées par d'autres fonctions de plus haut niveau, afin de pouvoir utiliser le "dispatch multiple" suivant le type d'entrées fournies.

Exemple avec la factorisation LU:

In [ ]:
T = Float64
n = 1000
A = rand(T, n, n)
@code_typed lu(A) # on voit l'appel à LinearAlgebra.LAPACK.getrf!

In [ ]:
T = Double64
n = 1000
A = rand(T, n, n)
@code_typed lu(A) # ici, appel à generic_lufact! pour les types non supportés par LAPACK.

**Exercice:**
Résolvez un système `Ax=b` dense aléatoire de taille 1000 où `A` est définie positive avec une factorisation de Cholesky en utilisant `cholesky` pour la factorisation et `ldiv!` pour la résolution du système.
Comparez les vitesses d'exécution de chaque fonction en `Float64` et `Float32` avec `@benchmark`, et mesurez les résidus obtenus. 


In [ ]:
T = Float64
n = 1000
A = rand(T, n, n)
A = A' * A + I
b = rand(T, n)
x = similar(b);

In [ ]:
@benchmark cholesky($A)

In [ ]:
F = cholesky(A)
@benchmark ldiv!($x, $F, $b)

In [ ]:
ldiv!(x, F, b)
norm(A * x - b)

In [ ]:
T = Float32
n = 1000
A = rand(T, n, n)
A = A' * A + I
b = rand(T, n)
x = similar(b);

In [ ]:
@benchmark cholesky($A)

In [ ]:
F = cholesky(A)
@benchmark ldiv!($x, $F, $b)

In [ ]:
ldiv!(x, F, b)
Float64(norm(A * x - b))

## III) Résolution de systèmes de point de selle par méthodes directes

Dans cette partie, on va voir comment résoudre différents problèmes menant à la résolution d'un système de point de selle en utilisant des factorisations.

Rappel: un système de point de selle s'écrit sous la forme

$$\begin{bmatrix} M & A^T \\ A & 0 \end{bmatrix}\begin{bmatrix} x \\ y \end{bmatrix} = \begin{bmatrix} b \\ c \end{bmatrix},$$
où $M \succ 0$.

1) Ecrire les conditions d'optimalité du problème

$$\arg \min_x \tfrac{1}{2}\|Ax-b\|_2^2, \quad A \in \mathbb{R}^{m \times n}, m \ge n$$.

Reformuler ces conditions d'optimalité sous la forme d'un système de point de selle.
Comparer le temps de résolution du système de point de selle (en utilisant une factorisation appropriée) avec le temps de résolution du problème de moindre carrés en utilisant la factorisation $QR$. 

**Solution**: les conditions d'optimalité sont:
$$A^T A x = A^T b.$$

Avec $r = b - Ax$, on a pour conditions d'optimalité:

$$\begin{bmatrix} I & A \\ A^T & 0 \end{bmatrix} \begin{bmatrix} r \\ x \end{bmatrix} = \begin{bmatrix} b \\ 0 \end{bmatrix}$$

In [ ]:
m, n = 1000, 800
A = rand(m, n)
b = rand(m);

In [ ]:
function solve_saddle_pt(A, b)
    m, n = size(A)
    K = [I A;
         A' zeros(n, n)]
    rhs = [b; zeros(n)]
    F = bunchkaufman(K)
    sol = F \ rhs
    return sol[1:m], sol[m+1:end]
end
r1, x1 = solve_saddle_pt(A, b)
println(norm(b- A * x1 - r1))

In [ ]:
function solve_qr(A, b)
    m, n = size(A)
    F = qr(A)
    x = F \ b # résout min ||Rx - Qᵀb|| en interne 
    return b - A *x, x
end
r2, x2 = solve_qr(A, b)
println(norm(b - A * x2 - r2))

In [ ]:
println(norm(x2 - x1))
println(norm(r2 - r1))

In [ ]:
@benchmark solve_saddle_pt($A, $b)

In [ ]:
@benchmark solve_qr($A, $b)

Verdict: QR est plus rapide! Mais pourquoi cet exercice alors?

Parce que certains systèmes structurés peuvent se reformuler comme un problème d'optimisation qui est beaucoup plus rapide à résoudre.

### IV) Pour aller plus loin: réduire les allocations pour un code plus performant

La plus grande source d'allocations lorsqu'on fait de l'algèbre linéaire numérique est souvent la création de vecteurs ou de matrices.
Si on utilise une boucle `for` ou `while` avec des allocations à chaque itérations de la boucle, cela peut ralentir beaucoup le code.

On peut mesurer le nombre d'allocations avec la macro `@allocated`.

NOTE: au premier appel d'une fonction, elle est compilée et cela entraine un plus grand temps d'éxécution et des allocations. Les appels suivants ne souffrent pas de ce problème.

Le "broadcasting" est souvent un moyen simple de réduire le nombre d'allocations.
Au lieu de par exemple sommer deux vecteurs et de stocker le résultat dans un nouveau vecteur, on utilise `.=` et `.+` (ou alors `@.` avant les opérations souhaitées) pour effectuer les opérations éléments par éléments et les stocker dans un vecteur pré-alloué. 


In [ ]:
@allocated ones(100)

In [ ]:
n = 1000
a, b, c = rand(n), rand(n), rand(n);

In [ ]:
@allocated d = a + b

In [ ]:
@allocated c .= a .+ b

In [ ]:
@allocated @. c = a + b

In [ ]:
@benchmark @. $c = $a + $b

Le broadcasting dans le cas précédent est une manière plus pratique d'écrire la fonction:

In [ ]:
function my_custom_sum!(c, a, b)
    n = length(c)
    @assert n == length(a) == length(b)
    for i=1:n
        c[i] = a[i] + b[i]
    end
    return c
end

Une deuxième source d'allocations qu'il est souvent facile d'éviter et l'usage de matrices/ vecteurs construits à partir d'autres matrices/vecteurs.
La fonction `view` et la macro `@views` permettent de contourner ce problème.

In [ ]:
@allocated dot(a[1:100], b[1:100])

In [ ]:
@allocated dot(view(a, 1:100), view(b, 1:100))

In [ ]:
@allocated @views dot(a[1:100], b[1:100])

NOTE: `view` et `@views` ne créent pas un nouveau vecteur contenant `a[1:100]`. Si vous avez besoin de `a[1:100]` sous forme de vecteur par la suite, alors il ne faut pas les utiliser, et privilégier une copie.

**Attention**: même si réduire le nombre d'allocations dans les boucles permet souvent de réduire les temps de calculs, dans certains cas il peut être préférable de garder un code qui alloue. La manière la plus sûre de regarder si la dimininution du nombre d'allocations a un effet positif est de mesurer le temps d'exécution de votre code, avec `@benchmark` par exemple.